In [1]:
import sympy as sp
from sympy import init_printing
init_printing()
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent))
from cores.dynamical_systems.unicycle_circle_following import UnicycleCircleFollowing
import torch
import numpy as np

In [2]:
path_radius = 10.0
linear_velocity = 2.0

In [3]:
system_torch = UnicycleCircleFollowing(path_radius=path_radius, linear_velocity=linear_velocity, dtype=torch.float32)

In [4]:
d_e, theta_e = sp.symbols(r'd_e \theta_e')
u = sp.symbols(r'u')
states = sp.Matrix([d_e, theta_e])
controls = sp.Matrix([u])

In [5]:
f1 = linear_velocity * sp.sin(theta_e)
f2 = linear_velocity/path_radius - linear_velocity*sp.cos(theta_e)/(path_radius - d_e) + u

f = sp.Matrix([f1, f2])

# Test f(x,u)

In [6]:
f_func = sp.lambdify((states, controls), f)
state_np = np.array([1., 2.], dtype=np.float32)
control_np = np.array([3.], dtype=np.float32)
print(f_func(state_np, control_np))


[[1.8185948]
 [3.2924771]]


In [7]:
state_torch = torch.tensor(state_np, dtype=torch.float32).unsqueeze(0)
control_torch = torch.tensor(control_np, dtype=torch.float32).unsqueeze(0)
f_torch = system_torch(state_torch, control_torch)
print(f_torch)

tensor([[1.8186, 3.2925]])


# Test df/dx

In [8]:
df_dx = f.jacobian(states)
df_dx_func = sp.lambdify((states, controls), df_dx)
print(df_dx_func(state_np, control_np))

[[ 0.         -0.83229363]
 [ 0.01027523  0.20206609]]


In [9]:
df_dx_torch = system_torch.f_dx(state_torch, control_torch)
print(df_dx_torch)

tensor([[[ 0.0000, -0.8323],
         [ 0.0103,  0.2021]]])


# Test df/du

In [10]:
df_du = f.jacobian(controls)
df_du_func = sp.lambdify((states, controls), df_du)
print(df_du_func(state_np, control_np))

[[0]
 [1]]


In [11]:
df_du_torch = system_torch.f_du(state_torch, control_torch)
print(df_du_torch)

tensor([[[0.],
         [1.]]])


# d^2f/dx^2

In [12]:
f1_dxdx = f1.diff(states).jacobian(states)
print(f1_dxdx)

Matrix([[0, 0], [0, -2.0*sin(\theta_e)]])


In [13]:
f2_dxdx = f2.diff(states).jacobian(states)
print(f2_dxdx)

Matrix([[-0.004*cos(\theta_e)/(1 - 0.1*d_e)**3, 0.02*sin(\theta_e)/(1 - 0.1*d_e)**2], [0.02*sin(\theta_e)/(1 - 0.1*d_e)**2, 2.0*cos(\theta_e)/(10.0 - d_e)]])
